In [4]:
#GUI 
import tkinter as tk
from tkinter import *
from tkinter import ttk
from tkinter import filedialog, messagebox, ttk
from tkinter.messagebox import showinfo
import pandas as pd
import glob
import csv 
import json
import pymongo
from pymongo import MongoClient
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sb
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, 
NavigationToolbar2Tk)



client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["airportsData"]
collection = db['UK']

#functions
def browseFiles():
    filenames = filedialog.askopenfilenames(initialdir = "/", title = "Select files", filetypes = (("csv files", "*.csv"),("all files", "*.*")))
    showinfo(title='Selected', message=filenames)
      
    b1.pack_forget()
    b3.pack()
        

    
def cleanAndSave():
    messagebox.askquestion("Are you sure?", "Are you sure you want to clean and save dataset?")
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client["airportsData"]
    collection = db['UK']
    
    path = '/Users/paulinamalek/Desktop/Masters/Module 2 AP/AP_Dataset'
    csv_files = glob.glob(path + "/*.csv")
    df_list = (pd.read_csv(file) for file in csv_files)
    
    big_df = pd.concat(df_list, axis=1, ignore_index=False)
    big_df.drop_duplicates(inplace =True)
    big_df_subset = big_df[['type','name','frequency_mhz','iso_country']]
    new_df = big_df_subset.dropna()
    new_df_uk = new_df.drop(new_df[(new_df['iso_country'] != 'GB')].index, inplace =False)
    new_df_uk.columns = ['type2', 'type1', 'name', 'frequency_mhz', 'iso_country']
    newdfB = new_df_uk[['type1', 'name', 'frequency_mhz', 'iso_country']]    
    uk_airports = newdfB.drop(newdfB[(newdfB['type1'] == 'closed')].index, inplace =False)
    new_columns = uk_airports[['type1','name','frequency_mhz']]
    newUk = new_columns.drop(new_columns[(new_columns['type1'] != 'small_airport') & (new_columns['type1'] != 'medium_airport') & (new_columns['type1'] != 'large_airport')].index, inplace=False)
    sorted_uk = newUk.sort_values('type1', ascending=True, inplace=False)
    
    data_dict = sorted_uk.to_dict("records")
    collection.insert_many(data_dict)
    
    b3.pack_forget()
    l3.pack()
    b5.pack()
    
    
def largeMean():
    data = pd.DataFrame(list(collection.find()))
    large_airports = data[data['type1'] == 'large_airport']
    meanData = large_airports['frequency_mhz'].mean()
    

    
def largeMode():
    data = pd.DataFrame(list(collection.find()))
    large_airports = data[data['type1'] == 'large_airport']
    modeData = large_airports['frequency_mhz'].mode()
    

        
def largeMedian():
    data = pd.DataFrame(list(collection.find()))
    large_airports = data[data['type1'] == 'large_airport']
    medianData = large_airports['frequency_mhz'].median()
   
    
    
        
def correlation():
    data = pd.DataFrame(list(collection.find()))
    small_airports = data[data['type1'] == 'small_airport']
    small_airports = small_airports.reset_index()
    sb.heatmap(small_airports[['frequency_mhz']])
    
        
def moreMean():
    data = pd.DataFrame(list(collection.find()))
    largFreq = data[data['frequency_mhz'] >100 ]
    meanFreq = largFreq['frequency_mhz'].mean()
    
def moreMode():
    data = pd.DataFrame(list(collection.find()))
    largFreq = data[data['frequency_mhz'] >100 ]
    modeFreq = largFreq['frequency_mhz'].mode()
   
        
        
def moreMedian():
    data = pd.DataFrame(list(collection.find()))
    largFreq = data[data['frequency_mhz'] >100 ]
    medianFreq = largFreq['frequency_mhz'].median()
    
        
    
def createGraph():
    data = pd.DataFrame(list(collection.find()))

    small_airports = data[data['type1'] == 'small_airport']
    new_columns = small_airports[['frequency_mhz']]
    sorted_small = new_columns.reset_index()
    sorted_small.plot.line(title= "Small airports communication frequency")
    

def openNewWindow():
    newWindow = Toplevel(root)
    newWindow.title("Outputs and visualisations")
    newWindow.geometry("500x400")

    frame1 = tk.LabelFrame(newWindow, text= "Small Airports", font=("Ariel 15 bold"))
    frame3 = tk.LabelFrame(newWindow, text= "Large Airports", font=("Ariel 15 bold"))
    frame4 = tk.LabelFrame(newWindow, text = 'Frequencies more than 100 mhz', font=("Ariel 15 bold"))
    frame2 = tk.LabelFrame(newWindow, text = "Are some frequencies used more than others?", font=("Ariel 15 bold"))
    frame1.pack(side= "top", fill="both", expand=True)
    frame3.pack(side = "top", fill="both", expand=True)
    frame4.pack(side = 'top', fill="both", expand=True)
    frame2.pack(side = "top", fill="both", expand=True)
    
    b6 = Button(frame1, text = "Create a graph", command= createGraph)
    b7 = Button(frame3, text = "Mean", command = largeMode)
    b8 = Button(frame3, text = "Mode", command = largeMode)
    b9 = Button(frame3, text = "Median", command= largeMedian)
    b10 =Button(frame2, text ="Correlation", command = correlation)
    b11 = Button(frame4,text = "Mean", command =moreMean)
    b12 = Button(frame4, text = "Mode", command = moreMode)
    b13 = Button(frame4, text = "Median", command = moreMedian)
    
    
    data = pd.DataFrame(list(collection.find()))
    b6.pack(side=TOP)
    b7.pack(side=TOP)
    b8.pack(side =TOP)
    b9.pack(side =TOP)
    b10.pack(side=TOP)
    b11.pack(side=TOP)
    b12.pack(side= TOP)
    b13.pack(side= TOP)
    
    

    
#main window for GUI app
root = Tk()
root.title("Load Files")
root.geometry("600x200")
root.resizable(0,0)




#labels
l1 = Label(root, text = "Upload your files and add them to MongoDB")
l1.config(anchor=CENTER)
l1.config(font=("Ariel 25 bold"))
l1.pack()

l2 = Label(root, text = "Your file")
l3 = Label(root, text = "Your data is saved in MongoDB!")



#buttons
b1 = Button(root, text = "Browse Files", command = browseFiles)
b1.pack()

b3 = Button(root, text = "Clean and save to MongoDB", command = cleanAndSave)
b5 = Button(root, text = "See visualisations", command = openNewWindow)








    
root.mainloop()



